This notebook now replaces everything we did in manually and semi-manully in the previous
two nodebooks. We will use

1. PyTorch Models for prediction
2. Autograd for gradient computation
3. PyTorch Loss for loss computation
4. PyTorch Optimizer for parameter updates

A PyTorch Pipeline has three steps:

1. Design a model (input_size, output_size, forward_pass)
2. Construct the loss and optimizer
3. Training loop
 - forward pass: compute prediction and loss
 - backward pass: gradients
 - update the weights
 
 For all the above pipeline steps, we use PyTorch classes and methods 

In [11]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Step 0. Prepare data

Create tensors from our numpy dataset returned from sklearn 

In [24]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
n_samples, n_features = X.shape

# Split traing and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale our features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Convert our training and test data to tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# reshape our y_ data
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


print(f'X_train shape: {X_train.shape}', type(X_train))
print(f'y_train shape: {y_train.shape}', type(y_train))
print(f'samples: {n_samples}; features: {n_features}')


X_train shape: torch.Size([455, 30]) <class 'torch.Tensor'>
y_train shape: torch.Size([455, 1]) <class 'torch.Tensor'>
samples: 569; features: 30


### Step 1. Design the model

Our model is a linear model of type `f(x) = wx + b` and we apply sigmoid function at the end to get a probability
between -1 and 1.

In [26]:
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1) # 1, is the output
        
    def forward(self, x):
        # apply sigmoid function
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
# define our PyTorch model, which is callable
model = LogisticRegression(n_features)

### Step 2. Construct loss and optimizer

In [27]:
# Since this is a classification, we going to use binariy cross entropy loss function
# and the same SGD optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Step 4. Train loop

In [33]:
num_epochs = 150
for epoch in range(num_epochs):
    # forward pass
    # Forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    # Backward pass and compute gradients
    loss.backward()
    
    # update our weights
    optimizer.step()
    
     # zero out the gradients computed in the backward pass as it sums
    # in a list property .grad
    optimizer.zero_grad()
    
    # print some status and progress
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch + 1},  loss: {loss.item():4f}')
        
# Let's evaluate the model and want to do it outside the computational graph
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc:.4f}')

epoch: 10,  loss: 0.120576
epoch: 20,  loss: 0.119965
epoch: 30,  loss: 0.119367
epoch: 40,  loss: 0.118783
epoch: 50,  loss: 0.118211
epoch: 60,  loss: 0.117650
epoch: 70,  loss: 0.117102
epoch: 80,  loss: 0.116565
epoch: 90,  loss: 0.116039
epoch: 100,  loss: 0.115524
epoch: 110,  loss: 0.115018
epoch: 120,  loss: 0.114523
epoch: 130,  loss: 0.114037
epoch: 140,  loss: 0.113561
epoch: 150,  loss: 0.113094
accuracy: 0.9737
